In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.2 MB/s eta 0:00:00


In [ ]:
from Bio import AlignIO
import pandas as pd



result = []

## загружаем элаймент
alignment = AlignIO.read("tcbfprs1.aln", "clustal")
print("Alignment length %i" % alignment.get_alignment_length())

# получаем словарь последовательность:мотив.
# Номера интересующих позиций надо будет найти в полученном элайменте.
# # Можно открыть элаймент в mega или snapgene и с
# помощью втроенного поиска найти нужные
# мотивы, а найдя мотивы посмотреть позиции
# интересующих остатков в элайменте

for record in alignment:
    new_motif = {'ProteinID':record.id,

                '17':str(record.seq[17:18]),
                '60':str(record.seq[60:61]),
                '61':str(record.seq[61:62]),

                '28':str(record.seq[28:29]),
                '59':str(record.seq[59:60]),
                '127':str(record.seq[127:128]),

                '15':str(record.seq[15:16]),
                '79':str(record.seq[79:80]),
                '99':str(record.seq[99:100]),

                '5':str(record.seq[5:6]),
                '116':str(record.seq[116:117])
                            }
    result.append(new_motif)

## переводим словарь в dataframe
motif1 = pd.DataFrame(result)

Alignment length 190


In [ ]:
motif1

,ProteinID,17,60,61,28,59,127,15,79,99,5,116
0,WP_175076441.1,D,D,E,K,S,M,R,K,R,E,R
1,WP_190627843.1,D,D,E,D,S,S,R,K,R,E,R
2,WP_191141365.1,D,D,E,K,S,M,R,K,R,E,R
3,WP_192701293.1,D,D,E,K,S,M,R,K,R,E,R
4,WP_195465562.1,D,D,E,K,S,M,R,K,R,E,R
...,...,...,...,...,...,...,...,...,...,...,...,...
193,WP_018886358.1,D,D,E,K,S,M,R,K,R,E,R
194,WP_024854782.1,D,D,E,R,S,M,R,K,R,E,R
195,WP_028778915.1,D,D,E,K,S,M,R,K,R,E,R
196,WP_038572800.1,D,D,E,K,S,M,R,K,R,E,R


# Mg coordination

In [ ]:
## статистика по позиции 17
motif1['17'].value_counts()

,count
17,
D,196
-,2


In [ ]:
motif1['60'].value_counts()

,count
60,
D,197
N,1


In [ ]:
motif1['61'].value_counts()

,count
61,
E,198


# 59-Activation center

In [ ]:
motif1['28'].value_counts()

,count
28,
K,148
R,43
A,2
-,2
D,1
E,1
G,1


In [ ]:
motif1['59'].value_counts()

,count
59,
S,198


In [ ]:
motif1['127'].unique()

array(['M', 'S', '-', 'A', 'G', 'L', 'I', 'Q', 'N', 'T'], dtype=object)

# Nucleotidyl transfer

In [ ]:
motif1['15'].value_counts()

,count
15,
R,191
-,3
P,1
H,1
V,1
Q,1


In [ ]:
motif1['79'].value_counts()

,count
79,
K,198


In [ ]:
motif1['99'].value_counts()

,count
99,
R,198


# Unknown function

In [ ]:
motif1['5'].value_counts()

,count
5,
E,193
-,4
Q,1


In [ ]:
motif1['115'].value_counts()

,count
115,
W,195
A,1
S,1
G,1


In [ ]:
# объединяем позиции и присваиваем классы
# active, non-active итд. Вам нужно понять, какие
# могут быть классы, исходя из разнообразия
# мотивов в последовательностях полимераз

def define_site(df):
  seq = "".join([df['28'], df['127']])
  if seq == 'KM' or seq == 'RM':
    return '(K|R)M_group'
  elif seq == 'KS' or seq == 'RS':
    return '(K|R)S_group'
  elif seq == 'K-' or 'R-':
    return '(K|R)-_group'
  elif seq == 'KG' or '':
    return 'KG_group'
  elif seq == 'RM':
    return 'RM_group'
  elif seq == 'RA':
    return 'RA_group'
  elif seq == 'RS':
    return 'RS_group'
  elif seq == 'RG':
    return 'RG_group'
  elif seq == 'AM':
    return 'AM_group'
  elif seq == 'AA':
    return 'AA_group'
  elif seq == 'AS':
    return 'AS_group'
  elif seq == 'A-':
    return 'A-_group'
  elif seq == 'AG':
    return 'AG_group'
  else:
    return 'other'


# # делаем в таблице с информацией о
# последовательнотсях столбик motif, куда
# записываем результат анализа для мотива в
# каждой последовательности


motif1['motif'] = motif1.apply(define_site, axis = 1)

# статистика по мотивам
motif1['motif'].value_counts()

,count
motif,
KM_group,102
RM_group,33
other,29
KS_group,14
K-_group,11
RS_group,3
KG_group,2
RA_group,2
A-_group,1


In [ ]:
group_counts = motif1['motif'].value_counts()
print(group_counts)

In [ ]:
# объединяем позиции и присваиваем классы
# active, non-active итд. Вам нужно понять, какие
# могут быть классы, исходя из разнообразия
# мотивов в последовательностях полимераз

def define_site(df):
  seq = "".join([df['28'], df['127']])
  if seq == 'KM' or seq == 'RM':
    return '(K|R)M_group'
  elif seq == 'KS' or seq == 'RS':
    return '(K|R)S_group'
  elif seq == 'K-' or seq == 'R-':
    return '(K|R)-_group'
  elif seq == 'KG' or seq == 'RG':
    return '(K|R)G_group'
  elif seq == 'AM':
    return 'AM_group'
  elif seq == 'AA':
    return 'AA_group'
  elif seq == 'AS':
    return 'AS_group'
  elif seq == 'A-':
    return 'A-_group'
  elif seq == 'AG':
    return 'AG_group'
  else:
    return 'other'


# # делаем в таблице с информацией о
# последовательнотсях столбик motif, куда
# записываем результат анализа для мотива в
# каждой последовательности


motif1['motif'] = motif1.apply(define_site, axis = 1)

# статистика по мотивам
motif1['motif'].value_counts()

,count
motif,
(K|R)M_group,135
other,31
(K|R)S_group,17
(K|R)-_group,11
(K|R)G_group,2
A-_group,1
AM_group,1


In [ ]:
resi_127 = motif1['127'].unique()
resi_28 = motif1['28'].unique()

In [ ]:
import numpy as np

In [ ]:
np.array(motif1["ProteinID"])

array(['WP_175076441.1', 'WP_190627843.1', 'WP_191141365.1',
       'WP_192701293.1', 'WP_195465562.1', 'WP_196008328.1',
       'WP_196616409.1', 'WP_198141211.1', 'WP_208914373.1',
       'WP_213618722.1', 'WP_214688127.1', 'WP_226838061.1',
       'WP_227614639.1', 'WP_233278032.1', 'WP_235122363.1',
       'WP_240874965.1', 'WP_243119025.1', 'WP_248925060.1',
       'WP_249105374.1', 'WP_249311924.1', 'WP_249900056.1',
       'WP_251407190.1', 'WP_251520536.1', 'WP_253345878.1',
       'WP_268313908.1', 'WP_268410844.1', 'WP_269881098.1',
       'WP_270246350.1', 'WP_270258036.1', 'WP_270877402.1',
       'WP_272050524.1', 'WP_278508211.1', 'WP_281250443.1',
       'WP_283770932.1', 'ABF91142.1', 'AFB75603.1', 'APR82134.1',
       'CAN96226.1', 'EEG25937.1', 'EOS35382.1', 'GIP11948.1',
       'GJM71831.1', 'GMX66011.1', 'HAU85797.1', 'HDR7922196.1',
       'HJI48757.1', 'HLD91558.1', 'HRW21710.1', 'HWA29149.1',
       'HZI15040.1', 'MBC5744690.1', 'MBC8135277.1', 'MBD5081109.1',
  

In [ ]:
motifs_dict = dict.fromkeys(np.array(motif1["ProteinID"]))

In [ ]:
motifs_dict

{'WP_175076441.1': None,
 'WP_190627843.1': None,
 'WP_191141365.1': None,
 'WP_192701293.1': None,
 'WP_195465562.1': None,
 'WP_196008328.1': None,
 'WP_196616409.1': None,
 'WP_198141211.1': None,
 'WP_208914373.1': None,
 'WP_213618722.1': None,
 'WP_214688127.1': None,
 'WP_226838061.1': None,
 'WP_227614639.1': None,
 'WP_233278032.1': None,
 'WP_235122363.1': None,
 'WP_240874965.1': None,
 'WP_243119025.1': None,
 'WP_248925060.1': None,
 'WP_249105374.1': None,
 'WP_249311924.1': None,
 'WP_249900056.1': None,
 'WP_251407190.1': None,
 'WP_251520536.1': None,
 'WP_253345878.1': None,
 'WP_268313908.1': None,
 'WP_268410844.1': None,
 'WP_269881098.1': None,
 'WP_270246350.1': None,
 'WP_270258036.1': None,
 'WP_270877402.1': None,
 'WP_272050524.1': None,
 'WP_278508211.1': None,
 'WP_281250443.1': None,
 'WP_283770932.1': None,
 'ABF91142.1': None,
 'AFB75603.1': None,
 'APR82134.1': None,
 'CAN96226.1': None,
 'EEG25937.1': None,
 'EOS35382.1': None,
 'GIP11948.1': None,
 'G

In [ ]:
initial_alignment = AlignIO.read("tcbfprs1.aln", "clustal")


In [ ]:
from Bio import AlignIO

pos_127 = 127
pos_28 = 28

# Load the alignment
initial_alignment = AlignIO.read("tcbfprs1.aln", "clustal")

# Iterate over the sequences in the alignment
for record in initial_alignment:
    seq = str(record.seq)
    motifs_dict[record.id] = f'Group_{seq[pos_28]}{seq[pos_127]}'

In [ ]:
motifs_dict

{'WP_175076441.1': 'Group_KM',
 'WP_190627843.1': 'Group_DS',
 'WP_191141365.1': 'Group_KM',
 'WP_192701293.1': 'Group_KM',
 'WP_195465562.1': 'Group_KM',
 'WP_196008328.1': 'Group_KS',
 'WP_196616409.1': 'Group_KM',
 'WP_198141211.1': 'Group_A-',
 'WP_208914373.1': 'Group_KM',
 'WP_213618722.1': 'Group_KM',
 'WP_214688127.1': 'Group_KM',
 'WP_226838061.1': 'Group_KM',
 'WP_227614639.1': 'Group_RM',
 'WP_233278032.1': 'Group_K-',
 'WP_235122363.1': 'Group_KM',
 'WP_240874965.1': 'Group_KM',
 'WP_243119025.1': 'Group_RM',
 'WP_248925060.1': 'Group_KM',
 'WP_249105374.1': 'Group_KM',
 'WP_249311924.1': 'Group_KM',
 'WP_249900056.1': 'Group_KM',
 'WP_251407190.1': 'Group_KM',
 'WP_251520536.1': 'Group_KM',
 'WP_253345878.1': 'Group_KM',
 'WP_268313908.1': 'Group_KM',
 'WP_268410844.1': 'Group_KM',
 'WP_269881098.1': 'Group_KM',
 'WP_270246350.1': 'Group_RS',
 'WP_270258036.1': 'Group_KM',
 'WP_270877402.1': 'Group_KM',
 'WP_272050524.1': 'Group_KM',
 'WP_278508211.1': 'Group_KM',
 'WP_281